## Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wandb
!wandb login

wandb: Currently logged in as: viai (use `wandb login --relogin` to force relogin)


In [ ]:
import os

DATE = "11-07"
MODEL_NAME = "YOLOX"
FRAMEWORK = "mmDetection"
DATA_TYPE = "Spacing"
TITLE = "_".join([DATE, DATA_TYPE, MODEL_NAME, FRAMEWORK])

BASE_WORK_DIR = "/content/drive/Shareddrives/VIAI_team/tw/model_test"
WORKING_DIR = os.path.join(BASE_WORK_DIR, TITLE)

if not os.path.exists(WORKING_DIR):
    os.makedirs(WORKING_DIR)

# weight와 log 파일 저장소
print(f"WORKING DIRECTORY: {WORKING_DIR}")

WORKING DIRECTORY: /content/drive/Shareddrives/VIAI_team/tw/model_test/11-07_Spacing_YOLOX_mmDetection


In [ ]:
WANDB_PROJECT = "[Spacing]TW_mmDet"
WANDB_NAME = "yolact_r50_1x8_coco"

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Sun Nov  7 12:59:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install mmcv-full
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

fatal: destination path 'mmdetection' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mmdet
creating build/bdist.linux-x86_64/egg/mmdet/models
creating build/bdist.linux-x86_64/egg/mmdet/models/backbones
copying build/lib/mmdet/models/backbones/resnest.py -> build/bdist.linux-x86_64/egg/mmdet/models/backbones
copying build/lib/mmdet/models/backbones/trident_resnet.py -> build/bdist.linux-x86_64/egg/mmdet/models/backbones
copying build/lib/mmdet/models/b

In [ ]:
!pip install albumentations>=0.3.2 --no-binary imgaug,albumentations

In [ ]:
import mmcv
from mmcv import Config

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


import cv2
import os.path as osp

## Pretrained model 다운로드

https://github.com/open-mmlab/mmdetection/tree/master/configs/yolox

In [ ]:
!cd mmdetection; mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
PRETRAINED_MODEL="https://download.openmmlab.com/mmdetection/v2.0/yolact/yolact_r50_1x8_coco/yolact_r50_1x8_coco_20200908-f38d58df.pth"
SAVE_PATH="/content/mmdetection/checkpoints/yolact_r50_1x8_coco_20200908-f38d58df.pth"

In [ ]:
!wget -O {SAVE_PATH} {PRETRAINED_MODEL}

--2021-11-07 12:59:48--  https://download.openmmlab.com/mmdetection/v2.0/yolact/yolact_r50_1x8_coco/yolact_r50_1x8_coco_20200908-f38d58df.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.187.62
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.187.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141516001 (135M) [application/octet-stream]
Saving to: ‘/content/mmdetection/checkpoints/yolact_r50_1x8_coco_20200908-f38d58df.pth’

/content/mmdetectio 100%[===================>] 134.96M  12.4MB/s    in 11s     

2021-11-07 13:00:00 (12.3 MB/s) - ‘/content/mmdetection/checkpoints/yolact_r50_1x8_coco_20200908-f38d58df.pth’ saved [141516001/141516001]



In [ ]:
# 기본 config, pretrained 모델의 checkpoint
config_file = '/content/mmdetection/configs/yolact/yolact_r50_1x8_coco.py'
checkpoint_file = SAVE_PATH

## Config 설정

In [ ]:
@DATASETS.register_module(force=True)
class AccidaSpacingDataset(CocoDataset):
    CLASSES = ('spacing', )

In [ ]:
cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

checkpoint_config = dict(interval=1)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
img_size = 550
model = dict(
    type='YOLACT',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=-1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        zero_init_residual=False,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=1,
        add_extra_convs='on_input',
        num_outs=5,
        upsample_cfg=dict(mode='bilinear')),
    bbox_head=dict(
        type='YOLACTHead',
        num_classes=80,
        in_channels=256

In [ ]:
cfg.dataset_type = 'AccidaSpacingDataset'
cfg.data_root = "/content/drive/Shareddrives/VIAI_team/data/accida_masked_only_dataset_v1/spacing"

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_prefix = 'train/images'
cfg.data.train.ann_file = 'spacing_train_coco.json'
cfg.data.train.pipeline = cfg.train_pipeline

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_prefix = 'valid/images'
cfg.data.val.ann_file = 'spacing_valid_coco.json'
cfg.data.val.pipeline = cfg.test_pipeline

cfg.model.bbox_head.num_classes = 1
cfg.model.mask_head.num_classes = 1
cfg.model.segm_head.num_classes = 1

cfg.work_dir = WORKING_DIR
cfg.load_from = checkpoint_file

In [ ]:
cfg.optimizer.lr = 0.01 / 8
cfg.lr_config.warmup = None
cfg.lr_config.policy = 'step'

cfg.log_config = dict(
    interval = 50,
    hooks=[
           dict(type='TextLoggerHook', interval = 50),
           dict(type='WandbLoggerHook', interval = 50,
           init_kwargs=dict(
               project=WANDB_PROJECT, entity="viai", name=WANDB_NAME
              ),
          )
    ]
)

cfg.runner.max_epochs = 36
cfg.evaluation.interval = 6
cfg.evaluation.metric = ['bbox','segm']
cfg.checkpoint_config.interval = 6

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

In [ ]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=0,
        interpolation=1,
        p=0.5),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
]

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor'))
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(256, 256),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            # dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

data = dict(train=dict(pipeline=train_pipeline))

In [ ]:
print(f'{cfg.pretty_text}')

checkpoint_config = dict(interval=6)
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook', interval=50),
        dict(
            type='WandbLoggerHook',
            interval=50,
            init_kwargs=dict(
                project='[Spacing]TW_mmDet',
                entity='viai',
                name='yolact_r50_1x8_coco'))
    ])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = '/content/mmdetection/checkpoints/yolact_r50_1x8_coco_20200908-f38d58df.pth'
resume_from = None
workflow = [('train', 1)]
img_size = 550
model = dict(
    type='YOLACT',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=-1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        zero_init_residual=False,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://r

## Train

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True, meta=dict())

2021-11-07 15:13:58,765 - mmdet - INFO - load checkpoint from /content/mmdetection/checkpoints/cascade_mask_rcnn_r50_fpn_mstrain_3x_coco_20210628_164719-5bdc3824.pth
2021-11-07 15:13:58,767 - mmdet - INFO - Use load_from_local loader


loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


2021-11-07 15:13:59,050 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.2.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.2.fc_cls.bias: copying 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2021-11-07 15:14:21,884 - mmdet - INFO - Epoch [1][50/337]	lr: 1.250e-03, eta: 3:15:03, time: 0.348, data_time: 0.058, memory: 4201, loss_rpn_cls: 0.0668, loss_rpn_bbox: 0.0122, s0.loss_cls: 0.0879, s0.acc: 98.3574, s0.loss_bbox: 0.0339, s0.loss_mask: 0.2203, s1.loss_cls: 0.0510, s1.acc: 96.1641, s1.loss_bbox: 0.0176, s1.loss_mask: 0.1350, s2.loss_cls: 0.0162, s2.acc: 98.3887, s2.loss_bbox: 0.0046, s2.loss_mask: 0.0607, loss: 0.7063
2021-11-07 15:14:36,575 - mmdet - INFO - Epoch [1][100/337]	lr: 1.250e-03, eta: 2:59:37, time: 0.294, data_time: 0.012, memory: 4201, loss_rpn_cls: 0.0302, los

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 82/82, 6.9 task/s, elapsed: 12s, ETA:     0s

...

DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.158
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.089
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | a

2021-11-07 18:06:32,615 - mmdet - INFO - Epoch(val) [96][82]	bbox_mAP: 0.0760, bbox_mAP_50: 0.1580, bbox_mAP_75: 0.0580, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0130, bbox_mAP_l: 0.0890, bbox_mAP_copypaste: 0.076 0.158 0.058 0.000 0.013 0.089, segm_mAP: 0.0020, segm_mAP_50: 0.0080, segm_mAP_75: 0.0000, segm_mAP_s: 0.0000, segm_mAP_m: 0.0000, segm_mAP_l: 0.0220, segm_mAP_copypaste: 0.002 0.008 0.000 0.000 0.000 0.022


DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.025
 Average Recall     (AR) @[ IoU=0.50:0.95 | a

2021-11-07 18:06:50,861 - mmdet - INFO - Epoch [97][50/337]	lr: 1.250e-05, eta: 0:06:01, time: 0.364, data_time: 0.063, memory: 4208, loss_rpn_cls: 0.0054, loss_rpn_bbox: 0.0044, s0.loss_cls: 0.0354, s0.acc: 98.6582, s0.loss_bbox: 0.0209, s0.loss_mask: 0.0522, s1.loss_cls: 0.0188, s1.acc: 98.6719, s1.loss_bbox: 0.0247, s1.loss_mask: 0.0295, s2.loss_cls: 0.0108, s2.acc: 98.5410, s2.loss_bbox: 0.0173, s2.loss_mask: 0.0157, loss: 0.2351
2021-11-07 18:07:06,110 - mmdet - INFO - Epoch [97][100/337]	lr: 1.250e-05, eta: 0:05:48, time: 0.305, data_time: 0.013, memory: 4208, loss_rpn_cls: 0.0054, loss_rpn_bbox: 0.0059, s0.loss_cls: 0.0356, s0.acc: 98.6738, s0.loss_bbox: 0.0217, s0.loss_mask: 0.0476, s1.loss_cls: 0.0199, s1.acc: 98.5371, s1.loss_bbox: 0.0270, s1.loss_mask: 0.0260, s2.loss_cls: 0.0107, s2.acc: 98.4316, s2.loss_bbox: 0.0163, s2.loss_mask: 0.0143, loss: 0.2305
2021-11-07 18:07:21,269 - mmdet - INFO - Epoch [97][150/337]	lr: 1.250e-05, eta: 0:05:34, time: 0.303, data_time: 0.012, me

learning_rate,████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,██▇▆▄▃▄▃▄▃▃▄▃▂▃▃▃▂▃▄▂▂▂▂▃▂▃▃▃▃▂▃▃▃▁▂▁▂▁▂
train/loss_rpn_bbox,▇▆██▃▅▃▅▆▄▄▄▃▃▃▁▇▄▆▆▂▄▄▄▆▂▆▂▃▄▄▃▃▄▂▄▃▆▃▄
train/loss_rpn_cls,█▅▄▄▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▂▂▁▁▁▁▁▁▂▂▂▁
train/s0.acc,▃▂▁▁▆▅▅▅▅▅▆▅▆▅▆▆▇▆▇▆▅▆▆▇▆▇█▆▇▆▇▆▆▆▇▇▇▇▆▇
train/s0.loss_bbox,███▇▂▃▄▃▃▃▃▄▃▄▃▃▂▂▂▃▂▂▃▁▂▁▁▃▂▂▂▃▂▂▁▂▁▁▁▁
train/s0.loss_cls,█▇▇▇▃▃▃▃▃▃▂▄▂▄▂▃▂▂▂▃▃▂▂▂▂▂▁▃▂▂▂▂▂▂▁▁▁▁▂▁
train/s0.loss_mask,█▇▄▅▄▃▃▂▄▃▄▄▃▁▃▃▃▃▅▅▃▂▂▄▅▃▄▂▄▃▂▃▄▄▂▃▂▃▂▂
train/s1.acc,▆▃▁▁▆▆▆▅▅▅▆▄▆▃▇▇▆▇▇▆▆▇▇█▆▇▇▆▆▅▆▇▅▇█▇▇█▇▇
train/s1.loss_bbox,▁▅██▅▄▇▄▆▅▅▆▅▇▅▅▄▅▄▅▄▄▅▃▄▅▅▆▅▇▅▇▅▄▁▄▂▃▃▄


## Inference

In [ ]:
checkpoint_file = os.path.join(WORKING_DIR, "latest.pth")
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

Use load_from_local loader


In [ ]:
test_images = []
for r, ds, fs in os.walk("/content/drive/Shareddrives/VIAI_team/data/accida_segmentation_dataset_v3_1/spacing/test/images"):
    for f in fs:
        test_images.append(os.path.join(r, f))

In [ ]:
import cv2

def show_result(model_ckpt, image_list, n=5):
    for i, image in enumerate(image_list):
        img = cv2.imread(image)
        result = inference_detector(model_ckpt, img)
        show_result_pyplot(model_ckpt, img, result, score_thr=0.5)
        if i > n:
            break

In [1]:
show_result(model_ckpt, test_images, n=10)